In [1]:
import os
os.getcwd()

'/home/virginia/Workspace/updated_AutoGP/autogp/kernels'

In [21]:
import numpy as np
import tensorflow as tf
import math

import kernel
from scipy.spatial import distance


In [22]:
def eye(N):
    return tf.diag(tf.ones(tf.stack([N, ]), dtype='float32'))


In [23]:
def euclidean(vector1, vector2):
    '''calculate the euclidean distance, no numpy
    input: numpy.arrays or lists
    return: euclidean distance
    '''
    dist = [(a - b)**2 for a, b in zip(vector1, vector2)]
    dist = float(math.sqrt(sum(dist)))
    return dist

In [24]:
def euclidean2(vector1, vector2):
    ''' use scipy to calculate the euclidean distance. '''
    dist = distance.euclidean(vector1, vector2)
    return dist

In [25]:
points1 = np.zeros((4,2))
points1[:,0]=[1,2,3,4]
points1[:,1]=[2,3,4,5]

points1
#points1.shape[0]

array([[ 1.,  2.],
       [ 2.,  3.],
       [ 3.,  4.],
       [ 4.,  5.]])

In [26]:
#points2 = np.zeros((3,2))
#points2[:,0]=[4,5,6]
#points2[:,1]=[5,6,7]
#points2.shape[0]

In [27]:
points2 = points1

In [28]:
zip(points1[0], points2[0])

[(1.0, 1.0), (2.0, 2.0)]

In [29]:
distance = euclidean2(points1[0], points2[0])


In [30]:
distance

0.0

In [101]:
points2.shape

(4, 2)

In [102]:
white = 1
std_dev = 1.0
lengthscale = 1.0
MAX_DIST = 1e8

In [103]:
type(MAX_DIST)

float

In [120]:
distances = euclidean(points1[1,:], points2[1,:])
distances

0.0

In [105]:
type(-math.exp(math.sqrt(3.0)))

float

In [123]:
math.exp(-math.sqrt(3.0)/lengthscale*distances)

1.0

In [124]:
clip = tf.clip_by_value(distances, 0.0, MAX_DIST)
type(clip)

tensorflow.python.framework.ops.Tensor

In [108]:
tf1 = tf.shape(points1)
tf1

<tf.Tensor 'Shape_11:0' shape=(2,) dtype=int32>

In [125]:
white_noise = white * eye(tf.shape(points1)[0])

In [126]:
white_noise

<tf.Tensor 'mul_209:0' shape=(?, ?) dtype=float32>

In [136]:
#kern_matrix = tf.zeros((tf.shape(points1)[0], tf.shape(points2)[0]))
kern_matrix = tf.zeros((len(points1), len(points2)))
kern_matrix

<tf.Tensor 'zeros_6:0' shape=(4, 4) dtype=float32>

In [137]:
kern_matrix.shape[1]

Dimension(4)

In [138]:
tf.multiply(math.sqrt(3.0),distances) + 1

<tf.Tensor 'add_80:0' shape=() dtype=float32>

In [139]:
output_list = []
for i in range(kern_matrix.shape[0]):
    for j in range(kern_matrix.shape[1]):
        distances = euclidean(points1[i,:], points2[j,:])
        distances = tf.clip_by_value(distances, 0.0, MAX_DIST) 
        first_term = tf.scalar_mul(std_dev, (1.0 ]):+ tf.scalar_mul(math.sqrt(3.0),distances)))
        second_term = tf.exp(tf.scalar_mul(-math.sqrt(3.0)/lengthscale, distances))
        output_list.append(tf.multiply(first_term,second_term))

In [127]:
kernel_matrix = tf.transpose(tf.reshape((tf.stack(output_list)), [points2.shape[0],points1.shape[0]]))
kernel_matrix


<tf.Tensor 'transpose_4:0' shape=(4, 4) dtype=float32>

In [128]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
print(kernel_matrix)
print kernel_matrix.eval(session=sess)

Tensor("transpose_4:0", shape=(4, 4), dtype=float32)
[[ 1.          0.29782081  0.0439721   0.00537288]
 [ 0.29782081  1.          0.29782081  0.0439721 ]
 [ 0.0439721   0.29782081  1.          0.29782081]
 [ 0.00537288  0.0439721   0.29782081  1.        ]]


In [130]:
#if points2 is None: 
#    points2 = points1
#    white_noise = white * util.eye(tf.shape(points1)[0])
#else:
#    white_noise = 0.0
white_noise = white * eye(tf.shape(points1)[0])
kernel_matrix = kernel_matrix + white_noise

In [131]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
print(kernel_matrix)
print kernel_matrix.eval(session=sess)

Tensor("add_78:0", shape=(4, 4), dtype=float32)
[[ 2.          0.29782081  0.0439721   0.00537288]
 [ 0.29782081  2.          0.29782081  0.0439721 ]
 [ 0.0439721   0.29782081  2.          0.29782081]
 [ 0.00537288  0.0439721   0.29782081  2.        ]]


In [174]:
points1 = points1 / lengthscale
points2 = points2 / lengthscale
print(points1,point2)

(array([[ 1.],
       [ 2.],
       [ 3.],
       [ 4.]]), array([[ 4.],
       [ 5.],
       [ 6.]]))


In [179]:
(points1 ** 2).shape

(4, 1)

In [182]:
tf.reduce_sum(points1 ** 2, 1) #aggrego per le colonne quindi mantengo le righe quindi faccio x1+x2+..
# per tutte le osservazioni

<tf.Tensor 'Sum_14:0' shape=(4,) dtype=float64>

In [183]:
tf.expand_dims(tf.reduce_sum(points1 ** 2, 1), 1)

<tf.Tensor 'ExpandDims_12:0' shape=(4, 1) dtype=float64>

In [189]:
magnitude_square1 = tf.expand_dims(tf.reduce_sum(points1 ** 2, 1), 1)
magnitude_square2 = tf.expand_dims(tf.reduce_sum(points2 ** 2, 1), 1)
magnitude_square2

<tf.Tensor 'ExpandDims_18:0' shape=(3, 1) dtype=float64>

In [186]:
distances = (magnitude_square1 - 2 * tf.matmul(points1, tf.transpose(points2)) + 
             tf.transpose(magnitude_square2))
distances

<tf.Tensor 'add_37:0' shape=(4, 3) dtype=float64>

In [ ]:
distances = (magnitude_square1 - 2 * tf.matmul(points1, tf.transpose(points2)) + 
             tf.transpose(magnitude_square2))
distances = tf.clip_by_value(distances, 0.0, MAX_DIST);

kern = ((std_dev ** 2) * tf.exp(-distances / 2.0))


In [194]:
magnitude_square1

<tf.Tensor 'ExpandDims_17:0' shape=(4, 1) dtype=float64>

In [192]:
tf.matmul(points1, tf.transpose(points2)) + 

<tf.Tensor 'add_38:0' shape=(4, 3) dtype=float64>

In [171]:
kern + white_noise

<tf.Tensor 'add_34:0' shape=(4, 3) dtype=float64>